In [40]:
import itk
import xarray as xr
import numpy as np
from numcodecs import Blosc, blosc
import zarr
import os
from glob import glob
from zipfile import ZipFile
import warnings

from itkwidgets import view, compare

In [6]:
if 'SCRATCH' in os.environ:
    os.chdir(os.environ['SCRATCH'])

In [7]:
# Downloaded locally from Globus 
# http://dx.doi.org/doi:10.18126/M2QM0Z
slices = glob('./rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/*.tiff')
slices.sort()
slices

['./rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00000.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00001.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00002.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00003.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00004.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00005.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00006.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00007.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00008.tiff',
 './rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.h5/rec_SFRR_2600_B0p2_00009.tiff',
 './rec20160318_1915

In [8]:
image = itk.imread(slices)

In [5]:
# view(image, units='μm')

In [9]:
print(itk.size(image))
print(itk.spacing(image))
print(itk.origin(image))

itkSize3 ([2560, 2560, 2160])
itkVectorD3 ([1, 1, 1])
itkPointD3 ([0, 0, 0])


In [10]:
# Available in ITK 5.1 RC 2 and later
image_da = itk.xarray_from_image(image)
image_da

<xarray.DataArray (z: 2160, y: 2560, x: 2560)>
array([[[129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        ...,
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129]],

       [[129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        ...,
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129]],

       [[129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        ...,
...
        ...,
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129]],

       [[129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        ...,
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129]],

       [[129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        ...,
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129],
        [129, 129, 129, ..., 129, 129, 129]]], dtype=uint8)
Coordinates:
  * x        (x) float64 0.0 1.0 2.0 3.0 ... 2.557e+03 2.558e+03 2.559e+03
  * y        (y) float64 0.0 1.0 2.0 3.0 ... 2.557e+03 2.558e+03 2.559e+03
  * z        (z) float64 0.0 1.0 2.0 3.0 ... 2.157e+03 2.158e+03 2.159e+03
Attributes:
    direction:  [[1. 0. 0.]\n [0. 1. 0.]\n [0. 0. 1.]]

In [11]:
units = 'μm'
image_da.attrs['units'] = units
image_da.attrs

{'direction': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'units': 'μm'}

In [12]:
# multi-resolution pyramid
pyramid = [image_da]
reduced = image
while not np.all(np.array(itk.size(reduced)) < 64):
    level = len(pyramid)
    shrink_factors = [2,]*3
    reduced = itk.bin_shrink_image_filter(reduced, shrink_factors=shrink_factors)
    reduced_da = itk.xarray_from_image(reduced)
    reduced_da.attrs['units'] = units
    print('level', level)
    print('origin', itk.origin(reduced))
    print('spacing', itk.spacing(reduced))
    print('size', itk.size(reduced))
    pyramid.append(reduced_da)

level 1
origin itkPointD3 ([0.5, 0.5, 0.5])
spacing itkVectorD3 ([2, 2, 2])
size itkSize3 ([1280, 1280, 1080])
level 2
origin itkPointD3 ([1.5, 1.5, 1.5])
spacing itkVectorD3 ([4, 4, 4])
size itkSize3 ([640, 640, 540])
level 3
origin itkPointD3 ([3.5, 3.5, 3.5])
spacing itkVectorD3 ([8, 8, 8])
size itkSize3 ([320, 320, 270])
level 4
origin itkPointD3 ([7.5, 7.5, 7.5])
spacing itkVectorD3 ([16, 16, 16])
size itkSize3 ([160, 160, 135])
level 5
origin itkPointD3 ([15.5, 15.5, 15.5])
spacing itkVectorD3 ([32, 32, 32])
size itkSize3 ([80, 80, 67])
level 6
origin itkPointD3 ([31.5, 31.5, 31.5])
spacing itkVectorD3 ([64, 64, 64])
size itkSize3 ([40, 40, 33])


In [13]:
dataset_name = 'rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6'
store_name = dataset_name + '.zarr'

In [14]:
image_ds = image_da.to_dataset(name=dataset_name)
image_ds

<xarray.Dataset>
Dimensions:                                                     (x: 2560, y: 2560, z: 2160)
Coordinates:
  * x                                                           (x) float64 0...
  * y                                                           (y) float64 0...
  * z                                                           (z) float64 0...
Data variables:
    rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6  (z, y, x) uint8 ...

In [15]:
store = zarr.DirectoryStore(store_name)

blosc.use_threads = False
compressor = Blosc(cname='zstd', clevel=5)
chunk_size = 64

In [16]:
image_ds.to_zarr(store,
                 mode='w',
                 compute=True,
                 encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

In [17]:
pyramid_group_paths = [""]
for level in range(1, len(pyramid)):
    pyramid_group_paths.append('level_{0}.zarr'.format(level))
pyramid_group_paths

['',
 'level_1.zarr',
 'level_2.zarr',
 'level_3.zarr',
 'level_4.zarr',
 'level_5.zarr',
 'level_6.zarr']

In [18]:
root = zarr.group(store)
root.attrs['_MULTISCALE_LEVELS'] = pyramid_group_paths
root.attrs['_SPATIAL_IMAGE'] = dataset_name

In [20]:
reduced = image
for level in range(1, len(pyramid)):
    print('level', level)
    shrink_factors = [2,]*3
    reduced = itk.bin_shrink_image_filter(reduced, shrink_factors=shrink_factors)
    reduced_da = itk.xarray_from_image(reduced)
    reduced_da.attrs['units'] = units
    ds = reduced_da.to_dataset(name=dataset_name)
    compressor = Blosc(cname='zstd', clevel=5)
    ds.to_zarr(store,
               mode='w',
               group=pyramid_group_paths[level],
               compute=True,
               encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

level 1
level 2
level 3
level 4
level 5
level 6


In [21]:
# After all modifications to the store are complete, consolidate the metadata so it is 'cloud-ready'.
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>

In [22]:
for level in range(1, len(pyramid)):
    print('level', level)
    store = zarr.DirectoryStore(store_name + '/' + pyramid_group_paths[level])
    # Also consolidate the metadata on the pyramid levels so they can be used independently
    zarr.consolidate_metadata(store)

level 1
level 2
level 3
level 4
level 5
level 6


In [32]:
ds = xr.open_zarr(store_name, group='level_2.zarr', consolidated=True)
ds

,Array,Chunk
Bytes,221.18 MB,262.14 kB
Shape,"(540, 640, 640)","(64, 64, 64)"
Count,901 Tasks,900 Chunks
Type,uint8,numpy.ndarray


In [33]:
da = ds[dataset_name]
da

,Array,Chunk
Bytes,221.18 MB,262.14 kB
Shape,"(540, 640, 640)","(64, 64, 64)"
Count,901 Tasks,900 Chunks
Type,uint8,numpy.ndarray


In [35]:
image = itk.image_from_xarray(da)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [47]:
array_view = itk.array_view_from_image(image)
array_view[int(array_view.shape[0]/2),
           int(array_view.shape[1]/2),
           int(array_view.shape[2]/2)] = 0

In [ ]:
view(image, size_limit_3d=[700,]*3)